In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf


In [2]:
# Load the cdata dataset that we saved in practical 1
cdata = pd.read_csv('./dataset/cdata.csv', index_col=0, encoding='gb2312')
# Now create a subset of flows
flows = [440100, 110000, 441900, 320500, 440300, 320200, 130600]
# from cdata, we want the 42 flows that start and end as one of these 7 flows, but not the 7 self flows
# The subset selection has 3 parts: 1) OrigCode in flows; 2) DestCode in flows; 3) Origcode not equal to DestCode.
cdatasub = cdata[cdata['Orig_code'].isin(flows) & cdata['Dest_code'].isin(flows) & (cdata['Orig_code'] != cdata['Dest_code'])]

In [3]:
#Origin - 原点名称
#Orig_code - 原点编码
#Destination - 目的地名称
#Dest_code - 目的地编码
#Flow - 流量
#vi1_origpop - 原点人口
#wj1_destpop - 目的地人口
#vi2_origunemp - 原点失业率
#wj2_destunemp - 目的地失业率
#vi3_origmedinc - 原点中位收入
#wj3_destmedinc - 目的地中位收入
#vi4_origpctrent - 原点租房比例
#wj4_destpctrent - 目的地租房比例


In [4]:
#print(cdata[(cdata['Orig_code'] != cdata['Dest_code'])])
#Make a pivot table to look at the flow matrix
cdatasubmat = pd.pivot_table(cdatasub,values='Flow',index ='Origin',columns='Destination',fill_value=0,aggfunc=sum,margins=True)
cdatasubmat


Destination,东莞,保定,北京,广州,无锡,深圳,苏州,All
Origin,,,,,,,,
东莞,0,0,0,795,0,91031,0,91826
保定,0,0,2248,0,0,0,0,2248
北京,0,1439,0,0,0,0,0,1439
广州,2128,0,0,0,0,13084,0,15212
无锡,0,0,0,0,0,0,10670,10670
深圳,4416,0,0,3617,0,0,0,8033
苏州,0,0,0,0,1958058,0,0,1958058
All,6544,1439,2248,4412,1958058,104115,10670,2087486


# 1. Production Constrained Model

1. $$T_{ij} = A_{i}O_{i}W_{j}^{\alpha}d_{ij}^{-\beta}$$
where

2. $$O_{i} = \sum_{j}T_{ij}$$
and

3. $$A_{i} = \frac{1}{\sum_{j}W_{j}^{\alpha}d_{ij}^{-\beta}}$$

In the production-constrained model, $O_{i}$ does not have a parameter as it is a known constraint, although it could be dissaggregated and receive a superscript index. $A_{i}$ is known as a _balancing factor_ and is a vector of values which relate to each origin, $i$, which do a similar job to $k$ in the unconstrained/total constrained model. Specifically, $A_{i}$ ensures that flow estimates from each origin sum to the known totals, $O_{i}$ rather than just the overall total. This constraint is given by equation 2 above.

Now at this point, we could calculate all of the $O_{i}$s and $A_{i}$s by hand for our sample system and then set about guessing/estimating the parameter values for the rest of the model, but as you might have already suspected from last time, we can use Python, Statsmodels and the Poisson GLM to do this work for us!

We set about re-specifying the Production-Constrained model as a Poisson regression model in a similar way to how we did before. We need to take logs of the right-hand side of equation and assume that these are logarithmially linked to the Poisson distributed mean ($\lambda_{ij}$) of the $T_{ij}$ variable. Equation 1 (above) then becomes:

4. $$\lambda_{ij} = \exp( \mu_{i} + \alpha \ln W_{j} - \beta \ln d_{ij} )$$





In [5]:
cdatasub = cdatasub.assign(log_wj2_destsal = lambda x: np.log(x['wj2_destunemp']))
cdatasub = cdatasub.assign(log_dist = lambda x: np.log(x['dist']))

In [6]:
# Here we specify a model with no intercept (given by the -1 in the formula)
# In practice this means that all AiOis are estimated against an intercept of zero.
# Including the interval would mean setting the first borough in OrigNewCode to the intercept
# and interpreting all other categories in relation to that, which is less useful but would still work.
formula = "Flow ~ Origin + log_wj2_destsal + log_dist -1"
prodSim = smf.glm(formula=formula, data = cdatasub, family = sm.families.Poisson()).fit()
prodSim.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                   Flow   No. Observations:                   10
Model:                            GLM   Df Residuals:                        1
Model Family:                 Poisson   Df Model:                            8
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -201.89
Date:                Fri, 26 Jul 2024   Deviance:                       294.81
Time:                        19:53:34   Pearson chi2:                     301.
No. Iterations:                     9   Pseudo R-squ. (CS):              1.000
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Origin[东莞]        -21.8782      0.226    -96.871      0.000     -22.321     -21.436
Origin[保定]         -4.4423      0.154    -28.919      0.000      -4.743      -4.141
Origin[北京]        -27.0305      0.252   -107.414      0.000     -27.524     -26.537
Origin[广州]        -15.6669      0.192    -81.536      0.000     -16.044     -15.290
Origin[无锡]         -9.1830      0.150    -61.411      0.000      -9.476      -8.890
Origin[深圳]        -20.1736      0.204    -98.712      0.000     -20.574     -19.773
Origin[苏州]        -18.7242      0.226    -82.913      0.000     -19.167     -18.282
log_wj2_destsal    27.5148      0.166    165.683      0.000      27.189      27.840
log_dist           -4.4889      0.036   -125.212      0.000      -4.559      -4.419
===================================================================================
"""

So, what do we have from this model?

The $\alpha$ parameter related to destination attractiveness is 2.0471

The $\beta$ distance decay parameter is -2.2423

The 'coef' for each origin (given here as OrigNewCode[_code_]) is the logged $A_{i}O_{i}$ value for that origin.

# 2.1 Model Estimates

Now at this point you will be wanting to know what affect the constraints have had on the estimates produced by the model, so let’s plug the parameters back into Equation 4 and take a look.

First we need some $O_{i}$ and $D_{j}$ columns to store the total in and out flows. We can do this by grouping the dataframe by the Origin and Destination codes, and summing up the 'Total' flows.

In [7]:
#create some Oi and Dj columns in the dataframe and store row and column totals in them:
# First get the origin sums and rename the column created
O_i = cdatasub.groupby('Origin')['Flow'].sum().to_frame()
O_i.rename(columns = {'Flow':'O_i'}, inplace=True)
# Now get the destination sums
D_j = cdatasub.groupby('Destination')['Flow'].sum().to_frame()
D_j.rename(columns = {'Flow':'D_j'}, inplace=True)

# Merge in O_i
cdatasub = cdatasub.merge(O_i,left_on='Origin',right_index=True)
# Merge in D_j
cdatasub = cdatasub.merge(D_j,left_on='Destination',right_index=True)

In [8]:
# Do the same with the prodSim parameter estimates, first create a dataframe of 
mu_i = prodSim.params.to_frame()
# Rename index values associated with geographies so I can merge the values with cdatasub
# This approach uses a regex pattern that replaces all values with '' unless they are in the set E0123456789
#mu_i.rename(index = dict(zip(mu_i.index[0:-2].values, mu_i.index[0:-2].str.replace(r'[^123456789]','').values)),inplace=True)
# An easier approach is to simply subset the strings using a list comprehension.
#mu_i.rename(index = dict(zip(mu_i.index[:-2].values,[x[12:-1] for x in mu_i.index[:-2].values])))

# 重置索引，将索引转换为列
# 只保留 Origin 开头的行，并移除前缀
mu_i = mu_i[mu_i.index.str.startswith('Origin')].copy()
mu_i.index = mu_i.index.str.replace('Origin\[|\]', '', regex=True)
# Set column name to mu_I
mu_i.rename(columns = {0:'mu_i'}, inplace=True)
#print(df_filtered)
# Now merge
cdatasub = cdatasub.merge(mu_i, left_on='Origin',right_index=True)
cdatasub.head()

,Origin,Orig_code,Destination,Dest_code,Flow,vi1_origpop,wj1_destpop,vi2_origunemp,wj2_destunemp,vi3_origmedinc,...,Longitude_orig,Latitude_orig,Longitude_dest,Latitude_dest,dist,log_wj2_destsal,log_dist,O_i,D_j,mu_i
id,,,,,,,,,,,,,,,,,,,,,
2,东莞,441900,深圳,440300,91031,4391673,2512952,5.74,6.12,780.64,...,113.906810,22.903042,113.930976,22.544679,39.925081,1.811562,3.687005,91826,104115,-21.878214
8,东莞,441900,广州,440100,795,4391673,368260,5.74,5.45,780.64,...,113.906810,22.903042,113.530991,23.229884,52.906389,1.695616,3.968524,91826,4412,-21.878214
19,广州,440100,深圳,440300,13084,2512952,1345717,6.12,5.17,509.97,...,113.530991,23.229884,113.930976,22.544679,86.510208,1.642873,4.460262,15212,104115,-15.666925
10,广州,440100,东莞,441900,2128,4391673,502593,5.74,4.42,780.64,...,113.530991,23.229884,113.906810,22.903042,52.906389,1.486140,3.968524,15212,6544,-15.666925
22,深圳,440300,广州,440100,3617,2512952,1225235,6.12,5.78,509.97,...,113.930976,22.544679,113.530991,23.229884,86.510208,1.754404,4.460262,8033,4412,-20.173629


In [9]:
# Save parameter estimates into variables
alpha = prodSim.params[7]
beta = prodSim.params[8]

# Now generate some rounded estimates by 
cdatasub['prodsimest'] = np.round(np.exp(cdatasub['mu_i'] + alpha * cdatasub['log_wj2_destsal'] + beta * cdatasub['log_dist']))

In [10]:
# Now look at the matrix of flows
pd.pivot_table(cdatasub,values='prodsimest',index ='Origin',columns='Destination',fill_value=0,aggfunc=sum,margins=True)

Destination,东莞,保定,北京,广州,无锡,深圳,苏州,All
Origin,,,,,,,,
东莞,0,0,0,1056,0,90770,0,91826.0
保定,0,0,2248,0,0,0,0,2248.0
北京,0,1439,0,0,0,0,0,1439.0
广州,1652,0,0,0,0,13560,0,15212.0
无锡,0,0,0,0,0,0,10670,10670.0
深圳,4814,0,0,3219,0,0,0,8033.0
苏州,0,0,0,0,1958058,0,0,1958058.0
All,6466,1439,2248,4275,1958058,104330,10670,2087486.0


In [11]:
# Compared to the original flows - sum across columns to see constraints working.
cdatasubmat

Destination,东莞,保定,北京,广州,无锡,深圳,苏州,All
Origin,,,,,,,,
东莞,0,0,0,795,0,91031,0,91826
保定,0,0,2248,0,0,0,0,2248
北京,0,1439,0,0,0,0,0,1439
广州,2128,0,0,0,0,13084,0,15212
无锡,0,0,0,0,0,0,10670,10670
深圳,4416,0,0,3617,0,0,0,8033
苏州,0,0,0,0,1958058,0,0,1958058
All,6544,1439,2248,4412,1958058,104115,10670,2087486


Here it is very easy to see the origin constraints working. The sum across all destinations (columns) for each origin (rows) is the same (subject to rounding - note Barking and Dagenham, Bexley, Brent and Camden are all off by 1) in the predicted trips matrix and the observed trips matrix. Essentially, $\sum_{j}T_{ij} = \sum_{j}\lambda_{ij} = O_{i}$ more or less holds, but the same is not true for destinations summing for all origins, for instance, Barking and Dagenham is predicted many more trips than in reality, and Camden is predicted far fewer. In this sense, $\sum_{i}T_{ij} = \sum_{i}\lambda_{ij} = D_{j}$

## 2.2 How does the attraction constrained model perform?

NB I've copied over the same function that we used in the last practical.

In [12]:
# Goodness of fit (functions borrowed from previous practical)

# Function to compute R^2
def calcR2(obs,est):
    return np.power(np.corrcoef(obs,est),2.0)[0][1]

# Function to compute RMSE
def calcRMSE(obs,est):
    return np.sqrt((np.power((obs - est),2.0)).mean())

print("R squared =", calcR2(cdatasub['Flow'],cdatasub['prodsimest']))
print("RMSE =", calcRMSE(cdatasub['Flow'],cdatasub['prodsimest']))

R squared = 0.999999734030966
RMSE = 301.0318919981735


# 2.2.3 A 'what if...' scenario

Now that we have calibrated our parameters and produced some estimates, we can start to play around with some what-if scenarios.

In a 'what if' scenario, we make the assumption that the parameters for alpha and beta are universal (that is - they don't change subject to circumstance), and we use this model as a basis for exploring different scenarios by changing other data in the model, such as the observed $W_{j}$ values, here median income at the destination (the attractive force), or the cost of travelling between two places. As we're using straight-line distance to equal cost here it doesn't make a great deal of sense to change that in our model though!

So, by way of example - What if the government invested loads of money into a new Car Plant in Barking and Dagenham and as a result, average wages increased from a mere £16,200 to £25,000. A far fetched scenario, but one that could make a good experiment.

In [13]:
# now let's try a what if scenario
# The scenario says if destination median income is 16200, set the new value to be 25000, otherwise leave it the same.
cdatasub = cdatasub.assign(wj3_destsalScenario = np.where(cdatasub['wj2_destunemp'] == 16200, 25000,cdatasub['wj2_destunemp']))
# Take the log of the new variable
cdatasub = cdatasub.assign(log_wj3_destsalScenario = lambda x: np.log(x['wj3_destsalScenario']))

In [14]:
# Now plug these new values into the model and see what happens
cdatasub['prodsimest_scenario'] = np.round(np.exp(cdatasub['mu_i'] + alpha * cdatasub['log_wj3_destsalScenario'] + beta * cdatasub['log_dist']))
# Here's the matrix
pd.pivot_table(cdatasub,values='prodsimest_scenario',index ='Origin',columns='Destination',fill_value=0,aggfunc=sum,margins=True)

Destination,东莞,保定,北京,广州,无锡,深圳,苏州,All
Origin,,,,,,,,
东莞,0,0,0,1056,0,90770,0,91826.0
保定,0,0,2248,0,0,0,0,2248.0
北京,0,1439,0,0,0,0,0,1439.0
广州,1652,0,0,0,0,13560,0,15212.0
无锡,0,0,0,0,0,0,10670,10670.0
深圳,4814,0,0,3219,0,0,0,8033.0
苏州,0,0,0,0,1958058,0,0,1958058.0
All,6466,1439,2248,4275,1958058,104330,10670,2087486.0
